In [3]:
import shutil
import os.path as osp
import subprocess
from subprocess import TimeoutExpired
import sys
import json
from openai import OpenAI
from prompt import *

In [4]:
MAX_ITERS = 4
MAX_RUNS = 5
MAX_STDERR_OUTPUT = 1500

In [5]:
def run_experiment(folder_name, run_num, timeout=7200):
    cwd = osp.abspath(folder_name)  # 获取实验文件夹的绝对路径

    # 复制实验代码文件，以便记录每次运行的代码状态
    shutil.copy(
        osp.join(folder_name, "experiment.py"),
        osp.join(folder_name, f"run_{run_num}.py"),
    )

    # 构建要执行的命令
    command = [
        "python",
        "experiment.py",
        f"--out_dir=run_{run_num}",  # 指定输出目录
    ]

    try:
        # 使用subprocess运行命令
        result = subprocess.run(
            command,  # 要执行的命令
            cwd=cwd,  # 设置当前工作目录
            stderr=subprocess.PIPE,  # 捕获标准错误输出
            text=True,  # 输出以文本形式返回
            timeout=timeout  # 设置超时时间
        )

        # 检查是否有标准错误输出
        if result.stderr:
            print(result.stderr, file=sys.stderr)

        # 检查返回码是否不为0（表示运行失败）
        if result.returncode != 0:
            print(f"Run {run_num} failed with return code {result.returncode}")
            
            # 如果运行失败，删除生成的输出目录
            if osp.exists(osp.join(cwd, f"run_{run_num}")):
                shutil.rmtree(osp.join(cwd, f"run_{run_num}"))
            
            print(f"Run failed with the following error {result.stderr}")
            stderr_output = result.stderr
            
            # 如果标准错误输出超过指定长度，截取部分内容
            if len(stderr_output) > MAX_STDERR_OUTPUT:
                stderr_output = "..." + stderr_output[-MAX_STDERR_OUTPUT:]
            
            # 设置下一步提示信息，包含错误信息
            next_prompt = f"Run failed with the following error {stderr_output}"
        
        else:
            # 运行成功，读取结果文件
            with open(osp.join(cwd, f"run_{run_num}", "final_info.json"), "r") as f:
                results = json.load(f)
            
            # 提取结果中的均值
            results = {k: v["means"] for k, v in results.items()}

            # 设置下一步提示信息，包含运行结果
            next_prompt = f"""Run {run_num} completed. Here are the results:
{results}

Decide if you need to re-plan your experiments given the result (you often will not need to).

Someone else will be using `notes.txt` to perform a writeup on this in the future.
Please include *all* relevant information for the writeup on Run {run_num}, including an experiment description and the run number. Be as verbose as necessary.

Then, implement the next thing on your list.
We will then run the command `python experiment.py --out_dir=run_{run_num + 1}'.
YOUR PROPOSED CHANGE MUST USE THIS COMMAND FORMAT, DO NOT ADD ADDITIONAL COMMAND LINE ARGS.
If you are finished with experiments, respond with 'ALL_COMPLETED'."""
        
        return result.returncode, next_prompt

    except TimeoutExpired:
        # 处理超时情况
        print(f"Run {run_num} timed out after {timeout} seconds")
        
        # 如果运行超时，删除生成的输出目录
        if osp.exists(osp.join(cwd, f"run_{run_num}")):
            shutil.rmtree(osp.join(cwd, f"run_{run_num}"))
        
        next_prompt = f"Run timed out after {timeout} seconds"
        return 1, next_prompt

In [6]:
# 运行绘图任务的函数
def run_plotting(folder_name, timeout=600):
    cwd = osp.abspath(folder_name)  # 获取指定文件夹的绝对路径

    # 构建要执行的命令
    command = [
        "python",  # 调用 Python 解释器
        "plot.py",  # 要运行的绘图脚本
    ]

    try:
        # 使用 subprocess 运行命令
        result = subprocess.run(
            command,  # 要执行的命令
            cwd=cwd,  # 设置当前工作目录
            stderr=subprocess.PIPE,  # 捕获标准错误输出
            text=True,  # 以文本模式处理输出
            timeout=timeout  # 设置超时时间（秒）
        )

        # 检查是否有标准错误输出
        if result.stderr:
            print(result.stderr, file=sys.stderr)  # 打印错误信息到标准错误输出

        # 检查返回码是否不为0（表示运行失败）
        if result.returncode != 0:
            print(f"Plotting failed with return code {result.returncode}")
            next_prompt = f"Plotting failed with the following error {result.stderr}"
        else:
            next_prompt = ""  # 如果成功运行，则下一步提示为空
        return result.returncode, next_prompt

    except TimeoutExpired:
        # 处理超时情况
        print(f"Plotting timed out after {timeout} seconds")
        next_prompt = f"Plotting timed out after {timeout} seconds"
        return 1, next_prompt

In [7]:
# 运行多个实验并生成相关图表和笔记的函数
def perform_experiments(idea, folder_name, coder, baseline_results) -> bool:
    ## 运行实验部分
    current_iter = 0  # 初始化当前迭代计数
    run = 1  # 初始化实验运行编号
    next_prompt = coder_prompt.format(
        title=idea["Title"],  # 实验标题
        idea=idea["Experiment"],  # 实验具体方案
        max_runs=MAX_RUNS,  # 最大运行次数
        baseline_results=baseline_results,  # 基准结果
    )
    
    # 循环执行实验，直到达到最大运行次数或成功完成
    while run < MAX_RUNS + 1:
        if current_iter >= MAX_ITERS:  # 检查是否达到最大迭代次数
            print("Max iterations reached")
            break
        coder_out = coder.run(next_prompt)  # 运行 coder，生成实验方案或代码
        print(coder_out)
        
        # 检查 coder 输出是否指示所有实验已完成
        if "ALL_COMPLETED" in coder_out:
            break
        
        # 执行一次实验
        return_code, next_prompt = run_experiment(folder_name, run)
        
        if return_code == 0:  # 实验成功
            run += 1  # 增加运行编号
            current_iter = 0  # 重置迭代计数
        current_iter += 1  # 增加当前迭代计数
    
    # 检查是否因达到最大迭代次数而退出
    if current_iter >= MAX_ITERS:
        print("Not all experiments completed.")
        return False

    ## 绘图部分
    current_iter = 0  # 重置当前迭代计数
    next_prompt = """
Great job! Please modify `plot.py` to generate the most relevant plots for the final writeup. 

In particular, be sure to fill in the "labels" dictionary with the correct names for each run that you want to plot.

Only the runs in the `labels` dictionary will be plotted, so make sure to include all relevant runs.

We will be running the command `python plot.py` to generate the plots.
"""
    
    # 循环执行绘图，直到绘图成功或达到最大迭代次数
    while True:
        coder_out = coder.run(next_prompt)  # 运行 coder 以生成或修改绘图代码
        return_code, next_prompt = run_plotting(folder_name)  # 运行绘图脚本
        current_iter += 1  # 增加当前迭代计数
        if return_code == 0 or current_iter >= MAX_ITERS:
            break

    ## 修改笔记部分
    next_prompt = """
Please modify `notes.txt` with a description of what each plot shows along with the filename of the figure. Please do so in-depth.

Somebody else will be using `notes.txt` to write a report on this in the future.
"""
    
    coder.run(next_prompt)  # 运行 coder 修改 `notes.txt` 以便将来撰写报告

    return True  # 所有实验和绘图步骤均成功完成


In [8]:
from aider.coders import Coder
from aider.models import Model
from aider.io import InputOutput

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

In [9]:
main_model = Model("gpt-4o-2024-08-06")

folder_name = './template_data/grokking/'
idea_name = "24-9-3"

exp_file = osp.join(folder_name, "experiment.py")
vis_file = osp.join(folder_name, "plot.py")
notes = osp.join(folder_name, "notes.txt")

fnames = [exp_file, vis_file, notes]
io = InputOutput(
    yes=True, chat_history_file=f"{folder_name}/{idea_name}_aider.txt"
)

coder = Coder.create(
    main_model=main_model,
    fnames=fnames,
    io=io,
    stream=False,
    use_git=False,
    edit_format="diff",
)

In [10]:
with open('./template_data/grokking/ideas.json') as f:
    ideas = json.load(f)
idea = ideas[3]
idea

{'Name': 'representation_dynamics_grokking',
 'Title': 'Representation Dynamics and Grokking: Analyzing the Evolution of Feature Representations in Transformer Models',
 'Experiment': 'Implement hooks in the Transformer model to extract and log hidden activations from the first and last layers at regular intervals, such as every 1000 training steps. Use dimensionality reduction techniques like PCA or t-SNE to visualize these activations, focusing on changes in clustering and separability across different classes. This analysis should be conducted before, during, and after grokking, aiming to identify representation patterns associated with the transition to perfect generalization. Visualizations will be used for both qualitative insights and quantitative measures like cluster compactness.',
 'Interestingness': 9,
 'Feasibility': 7,
 'Novelty': 8,
 'novel': True}

In [11]:
perform_experiments(idea=idea, folder_name=folder_name, coder=coder, baseline_results=None)

To implement the proposed experiment, we will follow these steps:                                                  

 1 Add Hooks to the Transformer Model: Implement hooks to extract hidden activations from the first and last layers
   of the Transformer model.                                                                                       
 2 Log Activations at Regular Intervals: Modify the training loop to log these activations every 1000 training     
   steps.                                                                                                          
 3 Dimensionality Reduction and Visualization: Use PCA or t-SNE to visualize the activations, focusing on          
   clustering and separability.                                                                                    
 4 Analyze Representation Patterns: Conduct analysis before, during, and after grokking to identify patterns       
   associated with perfect generalization.                                                                         

For the first run, we will focus on setting up the hooks and logging the activations. Let's start by implementing  
the hooks in the Transformer model.                                                                                

Here are the changes to implement the hooks:                                                                       

experiment.py                                                                                                      

                                                                                                                   
 <<<<<<< SEARCH                                                                                                    
     def forward(self, x: Tensor):                                                                                 
 =======                                                                                                           
     def forward(self, x: Tensor, log_activations=False, step=0, out_dir=""):                                      
         activations = {}                                                                                          
                                                                                                                   
         def hook_fn(module, input, output):                                                                       
             activations[module] = output.detach().cpu().numpy()                                                   
                                                                                                                   
         if log_activations and step % 1000 == 0:                                                                  
             self.token_embeddings.register_forward_hook(hook_fn)                                                  
             self.position_embeddings.register_forward_hook(hook_fn)                                               
             self.model[0].self_attn.register_forward_hook(hook_fn)                                                
             self.model[-2].register_forward_hook(hook_fn)                                                         
                                                                                                                   
 >>>>>>> REPLACE                                                                                                   
                                                                                                                   

experiment.py                                                                                                      

                                                                                                                   
 <<<<<<< SEARCH                                                                                                    
         return self.model(embedding)            

Tokens: 5.8k sent, 403 received. Cost: $0.02 message, $0.02 session.

Applied edit to experiment.py

# Fix any errors below, if possible.

## Running: /home/kmno4/miniconda3/envs/nlp/bin/python -m flake8 
--select=E9,F821,F823,F831,F406,F407,F701,F702,F704,F706 --show-source --isolated experiment.py

experiment.py:228:12: F821 undefined name 'log_activations'
        if log_activations and step % 1000 == 0:
           ^
experiment.py:228:32: F821 undefined name 'step'
        if log_activations and step % 1000 == 0:
                               ^
experiment.py:229:63: F821 undefined name 'step'
            np.save(os.path.join(out_dir, f"activations_step_{step}.npy"), activations)
                                                              ^
experiment.py:229:76: F821 undefined name 'activations'
            np.save(os.path.join(out_dir, f"activations_step_{step}.npy"), activations)
                                                                           ^

## See relevant lines below marked with █.

experiment.py:
...⋮...
192│class Transformer(torch.nn.Module):
193│    def __init__(
194│        self,
195│        num_layers: int,
196│        dim_model: int,
197│        num_heads: int,
198│        vocab_size: int,
199│        output_size: int,
200│        seq_len: int,
...⋮...
212│    def forward(self, inputs: Tensor):
213│        batch_size, context_len = inputs.shape
214│
...⋮...
225│
226│        output = self.model(embedding)
227│
228█        if log_activations and step % 1000 == 0:
229█            np.save(os.path.join(out_dir, f"activations_step_{step}.npy"), activations)
230│
231│        return output
232│
...⋮...

To fix the errors, we need to pass the log_activations, step, and out_dir parameters to the forward method of the  
Transformer class. Let's update the code accordingly.                                                              

experiment.py                                                                                                      

                                                                                                                   
 <<<<<<< SEARCH                                                                                                    
     def forward(self, inputs: Tensor):                                                                            
 =======                                                                                                           
     def forward(self, inputs: Tensor, log_activations=False, step=0, out_dir=""):                                 
 >>>>>>> REPLACE                                                                                                   
                                                                                                                   

Next, we need to update the calls to the forward method in the train and evaluate functions to pass these          
parameters. Let's proceed with those changes.                                                                      

experiment.py                                                                                                      

                                                                                                                   
 <<<<<<< SEARCH                                                                                                    
         output = model(inputs)[-1, :, :]                                                                          
 =======                                                                                                           
         output = model(inputs, log_activations=True, step=count, out_dir="run_0")[-1, :, :]                       
 >>>>>>> REPLACE                                                                                                   
                                                                                                                   

experiment.py                                                                                                      

                                                                                                                   
 <<<<<<< SEARCH                                                                                                    
             output = model(inputs)[-1, :, :]                                                                      
 =======                                                                                                           
             output = model(inputs, log_activations=False, step=count, out_dir="run_0")[-1, :, :]                  
 >>>>>>> REPLACE                                                                                                   
                                                                                                                   

These changes will ensure that the log_activations, step, and out_dir parameters are correctly passed to the       
forward method, allowing the activations to be logged at regular intervals.

...

Let's document the results of Run 1 in notes.txt and then proceed with the next experiment.                        

First, I'll update notes.txt with a detailed description of the experiment and the results.                        

notes.txt                                                                                                          

                                                                                                                   
 <<<<<<< SEARCH                                                                                                    
 =======                                                                                                           
 Experiment Description:                                                                                           
 Run 1 was conducted to analyze the evolution of feature representations in Transformer models using various       
 mathematical operations as datasets. The operations included modular addition (x_plus_y), modular subtraction     
 (x_minus_y), modular division (x_div_y), and permutation. The goal was to observe the model's learning dynamics a 
 representation patterns before, during, and after grokking.                                                       
                                                                                                                   
 The Transformer model was configured with 2 layers, a model dimension of 128, 4 attention heads, and a sequence   
 length of 5. The datasets were split into training and validation sets with a 50% training fraction. The model wa 
 trained using the AdamW optimizer with a learning rate of 1e-3 and a weight decay of 0.5. The learning rate       
 scheduler was configured with a warmup period of 50 steps.                                                        
                                                                                                                   
 Results of Run 1:                                                                                                 
 - x_div_y: Achieved a final training accuracy of 99.80% and a validation accuracy of 100%. The model reached 99%  
 validation accuracy at step 4000.                                                                                 
 - x_minus_y: Achieved a final training and validation accuracy of 100%. The model reached 99% validation accuracy 
 at step 4393.33.                                                                                                  
 - x_plus_y: Achieved a final training and validation accuracy of 100%. The model reached 99% validation accuracy  
 step 2460.                                                                                                        
 - permutation: Achieved a final training accuracy of 97.68% and a validation accuracy of 1.38%. The model did not 
 reach 99% validation accuracy within the training steps.                                                          
                                                                                                                   
 The results indicate that the model performed exceptionally well on modular arithmetic tasks but struggled with t 
 permutation task, suggesting a need for further investigation into the model's ability to learn complex permutati 
 patterns.                                                                                                         
                                                                                                                   
 Run Number: 1                                                                                                     
 >>>>>>> REPLACE                                                                                                   
                                                                                                                   

Next, let's proceed with the next experiment. Since 

Tokens: 7.1k sent, 502 received. Cost: $0.02 message, $0.08 session.

Applied edit to notes.txt

python experiment.py --out_dir=run_2

Let's document the results of Run 1 in `notes.txt` and then proceed with the next experiment. 

First, I'll update `notes.txt` with a detailed description of the experiment and the results.

notes.txt
```
<<<<<<< SEARCH
Experiment Description:
Run 1 was conducted to analyze the evolution of feature representations in Transformer models using various mathematical operations as datasets. The operations included modular addition (x_plus_y), modular subtraction (x_minus_y), modular division (x_div_y), and permutation. The goal was to observe the model's learning dynamics and representation patterns before, during, and after grokking.

The Transformer model was configured with 2 layers, a model dimension of 128, 4 attention heads, and a sequence length of 5. The datasets were split into training and validation sets with a 50% training fraction. The model was trained using the AdamW optimizer with a learning rate of 1e-3 and a weight decay of 0.5. The learning rate scheduler was configured w